https://www.reddit.com/r/LangChain/comments/13jd9wo/improving_the_quality_of_qa_with_pdf/

In [18]:
from dotenv import load_dotenv

In [19]:
if not load_dotenv():
    print("Cannot load .env file. Environment file is not exists or not readable")
    exit(1)

In [20]:
from langchain.document_loaders import PyPDFLoader

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document


In [22]:
import os
import glob
from typing import List

In [23]:
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    PyMuPDFLoader,
    PyPDFLoader
)

In [24]:
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PyPDFLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"})
}

In [25]:
def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1].lower()
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"Unsupported file extension '{ext}'")

In [26]:
from multiprocessing import Pool
from tqdm import tqdm

In [27]:
def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]: 
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.lower()}"), recursive=True)
            )
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.upper()}"), recursive=True)
        )
    
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]
    print(filtered_files)
    # multiprocessing
    with Pool(processes=os.cpu_count()) as pool:
        results = []
        # progress bar
        with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
            # apply function to each item in the iterable in parallel, pool.imap_unordered(task, items)
            for i, docs in enumerate(pool.imap_unordered(load_single_document, filtered_files)):
                results.extend(docs)
                pbar.update()

    return results

In [28]:
# Load environment variables
persist_directory = os.environ.get('PERSIST_DIRECTORY')
source_directory = os.environ.get('SOURCE_DIRECTORY', 'src_documents')
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')
chunk_size = 500
chunk_overlap = 50

In [29]:
# from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("src_documents\\DIALOGPT.pdf")
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap,
    length_function=len)

docs = text_splitter.split_documents(pages)

len(docs)
# len(pages)

89

In [30]:
if __name__ == "__main__":
    documents = load_documents(source_directory, [])
    print(documents)

['src_documents\\DIALOGPT.pdf', 'src_documents\\DIALOGPT.pdf']


Loading new documents:   0%|                              | 0/2 [00:00<?, ?it/s]

In [ ]:
def process_documents(ignored_files: List[str] = []) -> List[Document]:
    """
    Load documents and split in chunks
    """
    print(f"Loading documents from {source_directory}")
    documents = load_documents(source_directory, ignored_files)
    if not documents:
        print("No new documents to load")
        exit(0)
    print(f"Loaded {len(documents)} new documents from {source_directory}")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")
    return texts

In [ ]:
texts = process_documents()

Loading documents from src_documents


Loading new documents:   0%|                              | 0/1 [00:00<?, ?it/s]

In [ ]:
def does_vectorstore_exist(persist_directory: str, embeddings: HuggingFaceEmbeddings) -> bool:
    """
    Checks if vectorstore exists
    """
    # db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    # if not db.get()['documents']:
    #     return False
    return True

In [ ]:
from constants import CHROMA_SETTINGS
import chromadb

In [17]:
def main():
    # Create embeddings
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
    # Chroma client
    chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)

    if does_vectorstore_exist(persist_directory, embeddings):
        # Update and store locally vectorstore
        print(f"Appending to existing vectorstore at {persist_directory}")
        db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)
        collection = db.get()
        texts = process_documents([metadata['source'] for metadata in collection['metadatas']])
        print(f"Creating embeddings. May take some minutes...")
        db.add_documents(texts)
    else:
        # Create and store locally vectorstore
        print("Creating new vectorstore")
        texts = process_documents()
        print(f"Creating embeddings. May take some minutes...")
        db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS, client=chroma_client)
    db.persist()
    db = None

    print(f"Ingestion complete! You can now run privateGPT.py to query your documents")

In [19]:
main()

Appending to existing vectorstore at db
Loading documents from source_documents


Loading new documents: 0it [00:00, ?it/s]

No new documents to load
Loaded 0 new documents from source_documents
Split into 0 chunks of text (max. 500 tokens each)
Creating embeddings. May take some minutes...
[]
Ingestion complete! You can now run privateGPT.py to query your documents


: 